In [1]:
import yaml
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec

In [2]:
with open("jsonplaceholder_openapi.yaml") as f:
    raw_jsonplaceholder_api_spec = yaml.load(f, Loader=yaml.Loader)
jsonplaceholder_api_spec = reduce_openapi_spec(raw_jsonplaceholder_api_spec)
jsonplaceholder_api_spec

ReducedOpenAPISpec(servers=[{'url': 'https://jsonplaceholder.typicode.com'}], description='See https://jsonplaceholder.typicode.com/', endpoints=[('GET /posts', 'Returns all posts', {'description': 'Returns all posts', 'responses': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type': 'array', 'items': {'type': 'object', 'required': ['id', 'userId', 'title', 'completed'], 'properties': {'id': {'type': 'integer'}, 'userId': {'type': 'integer'}, 'title': {'type': 'string'}, 'completed': {'type': 'string'}}}}}}}}), ('GET /posts/{id}', 'Returns a post by id', {'description': 'Returns a post by id', 'parameters': [{'name': 'id', 'in': 'path', 'required': True, 'description': 'The user id.', 'schema': {'type': 'integer', 'format': 'int64'}}], 'responses': {'description': 'Successful response', 'content': {'application/json': {'schema': {'type': 'object', 'required': ['id', 'userId', 'title', 'completed'], 'properties': {'id': {'type': 'integer'}, 'userId'

In [3]:
with open("apisguru_openapi.yaml") as f:
    raw_apisguru_api_spec = yaml.load(f, Loader=yaml.Loader)
apisguru_api_spec = reduce_openapi_spec(raw_apisguru_api_spec)
apisguru_api_spec

ReducedOpenAPISpec(servers=[{'url': 'https://api.apis.guru/v2'}], description='Wikipedia for Web APIs. Repository of API definitions in OpenAPI format.\n**Warning**: If you want to be notified about changes in advance please join our [Slack channel](https://join.slack.com/t/mermade/shared_invite/zt-g78g7xir-MLE_CTCcXCdfJfG3CJe9qA).\nClient sample: [[Demo]](https://apis.guru/simple-ui) [[Repo]](https://github.com/APIs-guru/simple-ui)\n', endpoints=[('GET /providers.json', 'List all the providers in the directory\n', {'description': 'List all the providers in the directory\n', 'responses': {'description': 'OK', 'content': {'application/json': {'schema': {'type': 'object', 'properties': {'data': {'type': 'array', 'items': {'type': 'string', 'minLength': 1}, 'minItems': 1}}}}}}}), ('GET /{provider}.json', 'List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n', {'description': 'List all APIs in the directory for a particular

In [4]:
from langchain_community.agent_toolkits.openapi.spec import ReducedOpenAPISpec
from langchain_core.language_models import BaseLanguageModel
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import Tool

In [5]:
API_PLANNER_PROMPT = """You are a planner that plans a sequence of API calls to assist with user queries against an API.

You should:
1) evaluate whether the user query can be solved by the API documented below. If no, say why.
2) if yes, generate a plan of API calls and say what they are doing step by step.
3) If the plan includes a DELETE call, you should always return an ask from the User for authorization first unless the User has specifically asked to delete something.

You should only use API endpoints documented below ("Endpoints you can use:").
You can only use the DELETE tool if the User has specifically asked to delete something. Otherwise, you should return a request authorization from the User first.
Some user queries can be resolved in a single API call, but some will require several API calls.
The plan will be passed to an API controller that can format it into web requests and return the responses.

----

Here are some examples:

Fake endpoints for examples:
GET /user to get information about the current user
GET /products/search search across products
POST /users/{{id}}/cart to add products to a user's cart
PATCH /users/{{id}}/cart to update a user's cart
PUT /users/{{id}}/coupon to apply idempotent coupon to a user's cart
DELETE /users/{{id}}/cart to delete a user's cart

User query: tell me a joke
Plan: Sorry, this API's domain is shopping, not comedy.

User query: I want to buy a couch
Plan: 1. GET /products with a query param to search for couches
2. GET /user to find the user's id
3. POST /users/{{id}}/cart to add a couch to the user's cart

User query: I want to add a lamp to my cart
Plan: 1. GET /products with a query param to search for lamps
2. GET /user to find the user's id
3. PATCH /users/{{id}}/cart to add a lamp to the user's cart

User query: I want to add a coupon to my cart
Plan: 1. GET /user to find the user's id
2. PUT /users/{{id}}/coupon to apply the coupon

User query: I want to delete my cart
Plan: 1. GET /user to find the user's id
2. DELETE required. Did user specify DELETE or previously authorize? Yes, proceed.
3. DELETE /users/{{id}}/cart to delete the user's cart

User query: I want to start a new cart
Plan: 1. GET /user to find the user's id
2. DELETE required. Did user specify DELETE or previously authorize? No, ask for authorization.
3. Are you sure you want to delete your cart?
----

Here are endpoints you can use. Do not reference any of the endpoints above.

{endpoints}

----

User query: {query}
Plan:"""
API_PLANNER_TOOL_NAME = "api_planner"
API_PLANNER_TOOL_DESCRIPTION = f"Can be used to generate the right API calls to assist with a user query, like {API_PLANNER_TOOL_NAME}(query). Should always be called before trying to call the API controller."

In [6]:
def _create_api_planner_tool(
    api_spec: ReducedOpenAPISpec, llm: BaseLanguageModel
) -> Tool:
    from langchain.chains.llm import LLMChain

    endpoint_descriptions = [
        f"{name} {description}" for name, description, _ in api_spec.endpoints
    ]
    prompt = PromptTemplate(
        template=API_PLANNER_PROMPT,
        input_variables=["query"],
        partial_variables={"endpoints": "- " + "- ".join(endpoint_descriptions)},
    )
    print("- " + "- ".join(endpoint_descriptions))
    chain = LLMChain(llm=llm, prompt=prompt)
    tool = Tool(
        name=API_PLANNER_TOOL_NAME,
        description=API_PLANNER_TOOL_DESCRIPTION,
        func=chain.run,
    )
    return tool

In [7]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("azure_openai:gpt-4o", temperature=0.0)

In [8]:
_create_api_planner_tool(jsonplaceholder_api_spec, llm)

- GET /posts Returns all posts- GET /posts/{id} Returns a post by id


C:\Users\Admin\AppData\Local\Temp\ipykernel_35340\3604722709.py:15: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


Tool(name='api_planner', description='Can be used to generate the right API calls to assist with a user query, like api_planner(query). Should always be called before trying to call the API controller.', func=<bound method Chain.run of LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'endpoints': '- GET /posts Returns all posts- GET /posts/{id} Returns a post by id'}, template='You are a planner that plans a sequence of API calls to assist with user queries against an API.\n\nYou should:\n1) evaluate whether the user query can be solved by the API documented below. If no, say why.\n2) if yes, generate a plan of API calls and say what they are doing step by step.\n3) If the plan includes a DELETE call, you should always return an ask from the User for authorization first unless the User has specifically asked to delete something.\n\nYou should only use API endpoints documented below ("Endpoints you can use:").\nYou can only use 

In [9]:
_create_api_planner_tool(apisguru_api_spec, llm)

- GET /providers.json List all the providers in the directory
- GET /{provider}.json List all APIs in the directory for a particular providerName
Returns links to the individual API entry for each API.
- GET /{provider}/services.json List all serviceNames in the directory for a particular providerName
- GET /specs/{provider}/{api}.json Returns the API entry for one specific version of an API where there is no serviceName.- GET /specs/{provider}/{service}/{api}.json Returns the API entry for one specific version of an API where there is a serviceName.- GET /list.json List all APIs in the directory.
Returns links to the OpenAPI definitions for each API in the directory.
If API exist in multiple versions `preferred` one is explicitly marked.
Some basic info from the OpenAPI definition is cached inside each object.
This allows you to generate some simple views without needing to fetch the OpenAPI definition for each API.
- GET /metrics.json Some basic metrics for the entire directory.
Just

Tool(name='api_planner', description='Can be used to generate the right API calls to assist with a user query, like api_planner(query). Should always be called before trying to call the API controller.', func=<bound method Chain.run of LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'endpoints': '- GET /providers.json List all the providers in the directory\n- GET /{provider}.json List all APIs in the directory for a particular providerName\nReturns links to the individual API entry for each API.\n- GET /{provider}/services.json List all serviceNames in the directory for a particular providerName\n- GET /specs/{provider}/{api}.json Returns the API entry for one specific version of an API where there is no serviceName.- GET /specs/{provider}/{service}/{api}.json Returns the API entry for one specific version of an API where there is a serviceName.- GET /list.json List all APIs in the directory.\nReturns links to the OpenAPI def

In [10]:
from functools import partial
from typing import Any, Callable

from langchain_community.llms import OpenAI
from langchain_core.prompts import BasePromptTemplate

In [11]:
def _get_default_llm_chain(prompt: BasePromptTemplate) -> Any:
    from langchain.chains.llm import LLMChain

    return LLMChain(
        llm=OpenAI(),
        prompt=prompt,
    )


def _get_default_llm_chain_factory(
    prompt: BasePromptTemplate,
) -> Callable[[], Any]:
    """Returns a default LLMChain factory."""
    return partial(_get_default_llm_chain, prompt)

In [12]:
import json
from typing import cast

from langchain_community.tools.requests.tool import BaseRequestsTool
from langchain_core.tools import BaseTool
from pydantic import Field

In [13]:
MAX_RESPONSE_LENGTH = 5000


REQUESTS_GET_TOOL_DESCRIPTION = """Use this to GET content from a website.
Input to the tool should be a json string with 3 keys: "url", "params" and "output_instructions".
The value of "url" should be a string.
The value of "params" should be a dict of the needed and available parameters from the OpenAPI spec related to the endpoint.
If parameters are not needed, or not available, leave it empty.
The value of "output_instructions" should be instructions on what information to extract from the response,
for example the id(s) for a resource(s) that the GET request fetches.
"""

PARSING_GET_PROMPT = PromptTemplate(
    template="""Here is an API response:\n\n{response}\n\n====
Your task is to extract some information according to these instructions: {instructions}
When working with API objects, you should usually use ids over names.
If the response indicates an error, you should instead output a summary of the error.

Output:""",
    input_variables=["response", "instructions"],
)


REQUESTS_POST_TOOL_DESCRIPTION = """Use this when you want to POST to a website.
Input to the tool should be a json string with 3 keys: "url", "data", and "output_instructions".
The value of "url" should be a string.
The value of "data" should be a dictionary of key-value pairs you want to POST to the url.
The value of "output_instructions" should be instructions on what information to extract from the response, for example the id(s) for a resource(s) that the POST request creates.
Always use double quotes for strings in the json string."""

PARSING_POST_PROMPT = PromptTemplate(
    template="""Here is an API response:\n\n{response}\n\n====
Your task is to extract some information according to these instructions: {instructions}
When working with API objects, you should usually use ids over names. Do not return any ids or names that are not in the response.
If the response indicates an error, you should instead output a summary of the error.

Output:""",
    input_variables=["response", "instructions"],
)


REQUESTS_PATCH_TOOL_DESCRIPTION = """Use this when you want to PATCH content on a website.
Input to the tool should be a json string with 3 keys: "url", "data", and "output_instructions".
The value of "url" should be a string.
The value of "data" should be a dictionary of key-value pairs of the body params available in the OpenAPI spec you want to PATCH the content with at the url.
The value of "output_instructions" should be instructions on what information to extract from the response, for example the id(s) for a resource(s) that the PATCH request creates.
Always use double quotes for strings in the json string."""

PARSING_PATCH_PROMPT = PromptTemplate(
    template="""Here is an API response:\n\n{response}\n\n====
Your task is to extract some information according to these instructions: {instructions}
When working with API objects, you should usually use ids over names. Do not return any ids or names that are not in the response.
If the response indicates an error, you should instead output a summary of the error.

Output:""",
    input_variables=["response", "instructions"],
)


REQUESTS_PUT_TOOL_DESCRIPTION = """Use this when you want to PUT to a website.
Input to the tool should be a json string with 3 keys: "url", "data", and "output_instructions".
The value of "url" should be a string.
The value of "data" should be a dictionary of key-value pairs you want to PUT to the url.
The value of "output_instructions" should be instructions on what information to extract from the response, for example the id(s) for a resource(s) that the PUT request creates.
Always use double quotes for strings in the json string."""

PARSING_PUT_PROMPT = PromptTemplate(
    template="""Here is an API response:\n\n{response}\n\n====
Your task is to extract some information according to these instructions: {instructions}
When working with API objects, you should usually use ids over names. Do not return any ids or names that are not in the response.
If the response indicates an error, you should instead output a summary of the error.

Output:""",
    input_variables=["response", "instructions"],
)


REQUESTS_DELETE_TOOL_DESCRIPTION = """ONLY USE THIS TOOL WHEN THE USER HAS SPECIFICALLY REQUESTED TO DELETE CONTENT FROM A WEBSITE.
Input to the tool should be a json string with 2 keys: "url", and "output_instructions".
The value of "url" should be a string.
The value of "output_instructions" should be instructions on what information to extract from the response, for example the id(s) for a resource(s) that the DELETE request creates.
Always use double quotes for strings in the json string.
ONLY USE THIS TOOL IF THE USER HAS SPECIFICALLY REQUESTED TO DELETE SOMETHING."""

PARSING_DELETE_PROMPT = PromptTemplate(
    template="""Here is an API response:\n\n{response}\n\n====
Your task is to extract some information according to these instructions: {instructions}
When working with API objects, you should usually use ids over names. Do not return any ids or names that are not in the response.
If the response indicates an error, you should instead output a summary of the error.

Output:""",
    input_variables=["response", "instructions"],
)

In [14]:
class RequestsGetToolWithParsing(BaseRequestsTool, BaseTool):
    """Requests GET tool with LLM-instructed extraction of truncated responses."""

    name: str = "requests_get"
    """Tool name."""
    description: str = REQUESTS_GET_TOOL_DESCRIPTION
    """Tool description."""
    response_length: int = MAX_RESPONSE_LENGTH
    """Maximum length of the response to be returned."""
    llm_chain: Any = Field(
        default_factory=_get_default_llm_chain_factory(PARSING_GET_PROMPT)
    )
    """LLMChain used to extract the response."""

    def _run(self, text: str) -> str:
        from langchain.output_parsers.json import parse_json_markdown

        try:
            data = parse_json_markdown(text)
        except json.JSONDecodeError as e:
            raise e
        data_params = data.get("params")
        response: str = cast(
            str, self.requests_wrapper.get(data["url"], params=data_params)
        )
        response = response[: self.response_length]
        return self.llm_chain.predict(
            response=response, instructions=data["output_instructions"]
        ).strip()

    async def _arun(self, text: str) -> str:
        raise NotImplementedError()


class RequestsPostToolWithParsing(BaseRequestsTool, BaseTool):
    """Requests POST tool with LLM-instructed extraction of truncated responses."""

    name: str = "requests_post"
    """Tool name."""
    description: str = REQUESTS_POST_TOOL_DESCRIPTION
    """Tool description."""
    response_length: int = MAX_RESPONSE_LENGTH
    """Maximum length of the response to be returned."""
    llm_chain: Any = Field(
        default_factory=_get_default_llm_chain_factory(PARSING_POST_PROMPT)
    )
    """LLMChain used to extract the response."""

    def _run(self, text: str) -> str:
        from langchain.output_parsers.json import parse_json_markdown

        try:
            data = parse_json_markdown(text)
        except json.JSONDecodeError as e:
            raise e
        response: str = cast(str, self.requests_wrapper.post(data["url"], data["data"]))
        response = response[: self.response_length]
        return self.llm_chain.predict(
            response=response, instructions=data["output_instructions"]
        ).strip()

    async def _arun(self, text: str) -> str:
        raise NotImplementedError()


class RequestsPatchToolWithParsing(BaseRequestsTool, BaseTool):
    """Requests PATCH tool with LLM-instructed extraction of truncated responses."""

    name: str = "requests_patch"
    """Tool name."""
    description: str = REQUESTS_PATCH_TOOL_DESCRIPTION
    """Tool description."""
    response_length: int = MAX_RESPONSE_LENGTH
    """Maximum length of the response to be returned."""
    llm_chain: Any = Field(
        default_factory=_get_default_llm_chain_factory(PARSING_PATCH_PROMPT)
    )
    """LLMChain used to extract the response."""

    def _run(self, text: str) -> str:
        from langchain.output_parsers.json import parse_json_markdown

        try:
            data = parse_json_markdown(text)
        except json.JSONDecodeError as e:
            raise e
        response: str = cast(
            str, self.requests_wrapper.patch(data["url"], data["data"])
        )
        response = response[: self.response_length]
        return self.llm_chain.predict(
            response=response, instructions=data["output_instructions"]
        ).strip()

    async def _arun(self, text: str) -> str:
        raise NotImplementedError()


class RequestsPutToolWithParsing(BaseRequestsTool, BaseTool):
    """Requests PUT tool with LLM-instructed extraction of truncated responses."""

    name: str = "requests_put"
    """Tool name."""
    description: str = REQUESTS_PUT_TOOL_DESCRIPTION
    """Tool description."""
    response_length: int = MAX_RESPONSE_LENGTH
    """Maximum length of the response to be returned."""
    llm_chain: Any = Field(
        default_factory=_get_default_llm_chain_factory(PARSING_PUT_PROMPT)
    )
    """LLMChain used to extract the response."""

    def _run(self, text: str) -> str:
        from langchain.output_parsers.json import parse_json_markdown

        try:
            data = parse_json_markdown(text)
        except json.JSONDecodeError as e:
            raise e
        response: str = cast(str, self.requests_wrapper.put(data["url"], data["data"]))
        response = response[: self.response_length]
        return self.llm_chain.predict(
            response=response, instructions=data["output_instructions"]
        ).strip()

    async def _arun(self, text: str) -> str:
        raise NotImplementedError()


class RequestsDeleteToolWithParsing(BaseRequestsTool, BaseTool):
    """Tool that sends a DELETE request and parses the response."""

    name: str = "requests_delete"
    """The name of the tool."""
    description: str = REQUESTS_DELETE_TOOL_DESCRIPTION
    """The description of the tool."""

    response_length: int | None = MAX_RESPONSE_LENGTH
    """The maximum length of the response."""
    llm_chain: Any = Field(
        default_factory=_get_default_llm_chain_factory(PARSING_DELETE_PROMPT)
    )
    """The LLM chain used to parse the response."""

    def _run(self, text: str) -> str:
        from langchain.output_parsers.json import parse_json_markdown

        try:
            data = parse_json_markdown(text)
        except json.JSONDecodeError as e:
            raise e
        response: str = cast(str, self.requests_wrapper.delete(data["url"]))
        response = response[: self.response_length]
        return self.llm_chain.predict(
            response=response, instructions=data["output_instructions"]
        ).strip()

    async def _arun(self, text: str) -> str:
        raise NotImplementedError()

In [15]:
from typing import Any, List, Literal, Sequence

from langchain_community.utilities.requests import RequestsWrapper

In [16]:
Operation = Literal["GET", "POST", "PUT", "DELETE", "PATCH"]

PARSING_GET_PROMPT = PromptTemplate(
    template="""Here is an API response:\n\n{response}\n\n====
Your task is to extract some information according to these instructions: {instructions}
When working with API objects, you should usually use ids over names.
If the response indicates an error, you should instead output a summary of the error.

Output:""",
    input_variables=["response", "instructions"],
)

API_CONTROLLER_PROMPT = """You are an agent that gets a sequence of API calls and given their documentation, should execute them and return the final response.
If you cannot complete them and run into issues, you should explain the issue. If you're unable to resolve an API call, you can retry the API call. When interacting with API objects, you should extract ids for inputs to other API calls but ids and names for outputs returned to the User.


Here is documentation on the API:
Base url: {api_url}
Endpoints:
{api_docs}


Here are tools to execute requests against the API: {tool_descriptions}


Starting below, you should follow this format:

Plan: the plan of API calls to execute
Thought: you should always think about what to do
Action: the action to take, should be one of the tools [{tool_names}]
Action Input: the input to the action
Observation: the output of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I am finished executing the plan (or, I cannot finish executing the plan without knowing some other information.)
Final Answer: the final output from executing the plan or missing information I'd need to re-plan correctly.


Begin!

Plan: {input}
Thought:
{agent_scratchpad}
"""

In [17]:
def _create_api_controller_agent(
    api_url: str,
    api_docs: str,
    requests_wrapper: RequestsWrapper,
    llm: BaseLanguageModel,
    allow_dangerous_requests: bool,
    allowed_operations: Sequence[Operation],
) -> Any:
    from langchain.agents.agent import AgentExecutor
    from langchain.agents.mrkl.base import ZeroShotAgent
    from langchain.chains.llm import LLMChain

    tools: List[BaseTool] = []
    if "GET" in allowed_operations:
        get_llm_chain = LLMChain(llm=llm, prompt=PARSING_GET_PROMPT)
        tools.append(
            RequestsGetToolWithParsing(
                requests_wrapper=requests_wrapper,
                llm_chain=get_llm_chain,
                allow_dangerous_requests=allow_dangerous_requests,
            )
        )
    if "POST" in allowed_operations:
        post_llm_chain = LLMChain(llm=llm, prompt=PARSING_POST_PROMPT)
        tools.append(
            RequestsPostToolWithParsing(
                requests_wrapper=requests_wrapper,
                llm_chain=post_llm_chain,
                allow_dangerous_requests=allow_dangerous_requests,
            )
        )
    if "PUT" in allowed_operations:
        put_llm_chain = LLMChain(llm=llm, prompt=PARSING_PUT_PROMPT)
        tools.append(
            RequestsPutToolWithParsing(
                requests_wrapper=requests_wrapper,
                llm_chain=put_llm_chain,
                allow_dangerous_requests=allow_dangerous_requests,
            )
        )
    if "DELETE" in allowed_operations:
        delete_llm_chain = LLMChain(llm=llm, prompt=PARSING_DELETE_PROMPT)
        tools.append(
            RequestsDeleteToolWithParsing(
                requests_wrapper=requests_wrapper,
                llm_chain=delete_llm_chain,
                allow_dangerous_requests=allow_dangerous_requests,
            )
        )
    if "PATCH" in allowed_operations:
        patch_llm_chain = LLMChain(llm=llm, prompt=PARSING_PATCH_PROMPT)
        tools.append(
            RequestsPatchToolWithParsing(
                requests_wrapper=requests_wrapper,
                llm_chain=patch_llm_chain,
                allow_dangerous_requests=allow_dangerous_requests,
            )
        )
    if not tools:
        raise ValueError("Tools not found")
    prompt = PromptTemplate(
        template=API_CONTROLLER_PROMPT,
        input_variables=["input", "agent_scratchpad"],
        partial_variables={
            "api_url": api_url,
            "api_docs": api_docs,
            "tool_names": ", ".join([tool.name for tool in tools]),
            "tool_descriptions": "\n".join(
                [f"{tool.name}: {tool.description}" for tool in tools]
            ),
        },
    )
    print(prompt)
    agent = ZeroShotAgent(
        llm_chain=LLMChain(llm=llm, prompt=prompt),
        allowed_tools=[tool.name for tool in tools],
    )
    return AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True)

In [18]:
import re

In [19]:
API_CONTROLLER_TOOL_NAME = "api_controller"
API_CONTROLLER_TOOL_DESCRIPTION = f"Can be used to execute a plan of API calls, like {API_CONTROLLER_TOOL_NAME}(plan)."

In [20]:
def _create_api_controller_tool(
    api_spec: ReducedOpenAPISpec,
    requests_wrapper: RequestsWrapper,
    llm: BaseLanguageModel,
    allow_dangerous_requests: bool,
    allowed_operations: Sequence[Operation],
) -> Tool:
    """Expose controller as a tool.

    The tool is invoked with a plan from the planner, and dynamically
    creates a controller agent with relevant documentation only to
    constrain the context.
    """

    base_url = api_spec.servers[0]["url"]  # TODO: do better.

    def _create_and_run_api_controller_agent(plan_str: str) -> str:
        pattern = r"\b(GET|POST|PATCH|DELETE|PUT)\s+(/\S+)*"
        matches = re.findall(pattern, plan_str)
        endpoint_names = [
            "{method} {route}".format(method=method, route=route.split("?")[0])
            for method, route in matches
        ]
        docs_str = ""
        for endpoint_name in endpoint_names:
            found_match = False
            for name, _, docs in api_spec.endpoints:
                regex_name = re.compile(re.sub("\\{.*?\\}", ".*", name))
                if regex_name.match(endpoint_name):
                    found_match = True
                    docs_str += f"== Docs for {endpoint_name} == \n{yaml.dump(docs)}\n"
            if not found_match:
                raise ValueError(f"{endpoint_name} endpoint does not exist.")

        print('========================docs_str:',docs_str)
        agent = _create_api_controller_agent(
            base_url,
            docs_str,
            requests_wrapper,
            llm,
            allow_dangerous_requests,
            allowed_operations,
        )
        print(plan_str)
        return agent.run(plan_str)

    return Tool(
        name=API_CONTROLLER_TOOL_NAME,
        func=_create_and_run_api_controller_agent,
        description=API_CONTROLLER_TOOL_DESCRIPTION,
    )

In [21]:
# from langchain.chat_models import init_chat_model

# llm = init_chat_model("azure_openai:gpt-4o", temperature=0.0)

In [22]:
from langchain_community.utilities.requests import RequestsWrapper

requests_wrapper = RequestsWrapper()
ALLOW_DANGEROUS_REQUEST = True
allowed_operations: Sequence[Operation] = ("GET", "POST")

In [23]:
_create_api_controller_tool(
    jsonplaceholder_api_spec,
    requests_wrapper,
    llm,
    ALLOW_DANGEROUS_REQUEST,
    allowed_operations,
)

Tool(name='api_controller', description='Can be used to execute a plan of API calls, like api_controller(plan).', func=<function _create_api_controller_tool.<locals>._create_and_run_api_controller_agent at 0x000002C691103C40>)

In [24]:
_create_api_controller_tool(
    apisguru_api_spec,
    requests_wrapper,
    llm,
    ALLOW_DANGEROUS_REQUEST,
    allowed_operations,
)

Tool(name='api_controller', description='Can be used to execute a plan of API calls, like api_controller(plan).', func=<function _create_api_controller_tool.<locals>._create_and_run_api_controller_agent at 0x000002C691112B60>)

In [25]:
tools = [
    _create_api_planner_tool(jsonplaceholder_api_spec, llm),
    _create_api_controller_tool(
        jsonplaceholder_api_spec,
        requests_wrapper,
        llm,
        ALLOW_DANGEROUS_REQUEST,
        allowed_operations,
    ),
]

- GET /posts Returns all posts- GET /posts/{id} Returns a post by id


In [26]:
openapi_agent = init_chat_model("azure_openai:gpt-4o", temperature=0.0).bind_tools(
    tools
)

In [27]:
result = openapi_agent.invoke(
    "make me a playlist with the first song from kind of blue. call it machine blues."
)

In [28]:
result.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  api_planner (call_xf2odIt62nkXdWEOWIuQ5gQc)
 Call ID: call_xf2odIt62nkXdWEOWIuQ5gQc
  Args:
    __arg1: create a playlist named 'Machine Blues' with the first song from 'Kind of Blue' by Miles Davis.
  api_planner (call_kKJvCALb8SZdTTAD5TbMlE8B)
 Call ID: call_kKJvCALb8SZdTTAD5TbMlE8B
  Args:
    __arg1: get the first song from 'Kind of Blue' by Miles Davis.


In [29]:
from langchain.chains.llm import LLMChain

get_llm_chain = LLMChain(llm=llm, prompt=PARSING_GET_PROMPT)
test_tool = RequestsGetToolWithParsing(
    requests_wrapper=requests_wrapper,
    llm_chain=get_llm_chain,
    allow_dangerous_requests=True,
)

result = test_tool.invoke(
    {
        "text": """```json
{
  "url": "https://api.apis.guru/v2/providers.json",
  "params": {},
  "output_instructions": "Extract the list of provider names."
}
```"""
    }
)
print(result)

1forge  
1password  
6-dot-authentiqio  
ably  
abstractapi  
adafruit  
adobe  
adyen  
afterbanks  
agco-ats  
aiception  
airbyte  
airport-web  
akeneo  
alertersystem  
amadeus  
amazonaws  
amentum  
anchore  
apache  
apacta  
api.ebay  
api.gov  
api.video  
api2cart  
api2pdf  
apicurio  
apidapp  
apideck  
apigee  
apimatic  
apis.guru  
apisetu  
apispot  
apiz.ebay  
appcenter  
apple  
apptigent  
appveyor  
appwrite  
archive  
arespass  
art19  
asana  
asuarez  
atlassian  
ato  
aucklandmuseum  
authentiq  
autodealerdata  
autotask  
avaza  
aviationdata  
axesso  
azure  
balldontlie  
bandsintown  
bbc  
bbci  
bclaws  
beanstream  
beezup  
betfair  
bethmardutho  
bhagavadgita  
biapi  
bigdatacloud  
bigoven  
bigredcloud  
bikewise  
billbee  
billingo  
bintable  
bitbucket  
biztoc  
blazemeter  
bluemix  
botify  
botschaft  
box  
brainbi  
brandlovers  
braze  
brex  
bridgedb  
britbox  
browshot  
bufferapp  
bulksms  
bungie  
bunq  
byautomata  
c19qrs

In [30]:
len(result)

2717